<a href="https://colab.research.google.com/github/stuart-lane/MachineLearning/blob/main/FloatPrecision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import random
import math

import scipy
from scipy.stats import norm as normal
import sklearn

from scipy.optimize import minimize
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import torch
from torch.optim import Adam
from torch.optim import LBFGS

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

np.random.seed(123456)

DATA GENERATING PROCESS

In [2]:
n = 1000

beta0 = 0
beta1 = 1

x = np.random.randn(n)
u = np.random.logistic(0, 1, n)

ystar = beta0 + beta1 * x + u
y = (ystar > 0).astype(int)

u = u.reshape(-1, 1)
x = np.hstack((np.ones((n, 1)), x.reshape(-1, 1)))
y = y.reshape(-1, 1)

LOG-LIKELIHOOD FUNCTION

In [3]:
def negative_log_likelihood(beta, X, y):
    if isinstance(X, np.ndarray):
        xbhat = np.dot(X, beta)
        yhat = 1 / (1 + np.exp(-xbhat))
        loss = -np.sum(y * np.log(yhat) + (1 - y) * np.log(1 - yhat))
    elif isinstance(X, torch.Tensor):
        xbhat = torch.mm(X, beta)
        yhat = torch.sigmoid(xbhat)
        loss = -torch.sum(y * torch.log(yhat) + (1 - y) * torch.log(1 - yhat))
    else:
        raise ValueError("Unsupported input type. Use either numpy array or torch tensor.")
    return loss

In [ ]:
n = 1000

beta0 = 0
beta1 = 1

x = np.random.randn(n)
u = np.random.logistic(0, 1, n)

ystar = beta0 + beta1 * x + u
y = (ystar > 0).astype(int)

u = u.reshape(-1, 1)
x = np.hstack((np.ones((n, 1)), x.reshape(-1, 1)))
y = y.reshape(-1, 1)

LOG-LIKELIHOOD FUNCTION

In [ ]:
def negative_log_likelihood(beta, X, y):
    if isinstance(X, np.ndarray):
        xbhat = np.dot(X, beta)
        yhat = 1 / (1 + np.exp(-xbhat))
        loss = -np.sum(y * np.log(yhat) + (1 - y) * np.log(1 - yhat))
    elif isinstance(X, torch.Tensor):
        xbhat = torch.mm(X, beta)
        yhat = torch.sigmoid(xbhat)
        loss = -torch.sum(y * torch.log(yhat) + (1 - y) * torch.log(1 - yhat))
    else:
        raise ValueError("Unsupported input type. Use either numpy array or torch tensor.")
    return loss

DATA GENERATING PROCESS

In [ ]:
n = 1000

beta0 = 0.5
beta1 = 3
sigma = 1
beta = np.array([beta0, beta1])

x = np.random.randn(n)
u = np.random.logistic(0, 1, n)

y = beta0 + beta1 * x + u

u = u.reshape(-1, 1)
x = np.hstack((np.ones((n, 1)), x.reshape(-1, 1)))
y = y.reshape(-1, 1)

In [ ]:
def negative_log_likelihood(beta, X, y):
    if isinstance(X, np.ndarray):
        xbeta = np.dot(X, beta)
        expxbeta = np.exp(-y + xbeta)
        loss = -np.sum((-(y - xbeta) -  2 * np.log(1 + expxbeta)))
    elif isinstance(X, torch.Tensor):
        xbeta = torch.mm(X, beta)
        expxbeta = torch.exp(-y + xbeta)
        loss = -torch.sum(-(y - xbeta) -  2 * torch.log(1 + expxbeta))
    else:
        raise ValueError("Unsupported input type. Use either numpy array or torch tensor.")
    return loss

DATA GENERATING PROCESS

In [ ]:
n = 1000

beta0 = 0
beta1 = 1
sigma = 1
beta = np.array([beta0, beta1])

x = np.random.randn(n)
u = np.random.normal(0, sigma, n)

y_cutoff = 0

ystar = beta0 + beta1 * x + u
yind = (ystar > y_cutoff).astype(int)
y = yind * ystar

u = u.reshape(-1, 1)
x = np.hstack((np.ones((n, 1)), x.reshape(-1, 1)))
y = y.reshape(-1, 1)

LOG-LIKELIHOOD FUNCTION

In [ ]:
def normal_pdf(X, mu, sigma = 1):
    if isinstance(X, np.ndarray):
        constant = 1.0 / np.sqrt(2 * math.pi * sigma**2)
        exponent = -((X - mu)**2) / (2 * sigma**2)
        return constant * math.exp(exponent)
    elif isinstance(X, torch.Tensor):
        constant = 1.0 / torch.sqrt(2 * torch.tensor(math.pi) * sigma**2)
        exponent = -((X - mu)**2) / (2 * sigma**2)
        return constant * torch.exp(exponent)
    else:
        raise ValueError("Unsupported input type. Use either numpy array or torch tensor.")
    return loss


def negative_log_likelihood(beta, X, y, yind, y_cutoff, sigma = 1):
    if isinstance(X, np.ndarray):
        xbeta = np.dot(X, beta)
        term1 = (y - xbeta) / sigma
        normpdf = normal_pdf(term1, 0, 1) / sigma
        cdf_term2 = normal.cdf((xbeta - y_cutoff) / sigma)
        loss = - (np.sum(yind * np.log(normpdf) + (1 - yind) * np.log(1 - (cdf_term2))))
    elif isinstance(X, torch.Tensor):
        xbeta = torch.matmul(X, beta)
        term1 = (y - xbeta) / sigma
        normpdf = normal_pdf(term1, 0, 1) / sigma
        cdf_term2 = (xbeta - y_cutoff) / sigma
        loss = - (torch.sum(yind * torch.log(normpdf) + (1 - yind) * torch.log(1 - cdf_term2)))
    else:
        raise ValueError("Unsupported input type. Use either numpy array or torch tensor.")
    return loss

DATA FORMATTING

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 123)

train_df_keys = {'constant': x_train[:,0].flatten(), 'X1': x_train[:,1].flatten(), 'y': y_train.flatten()}
train_df = pd.DataFrame(train_df_keys)

In [6]:
# Convert the variables to torch tensors
x = torch.tensor(x_train, dtype = torch.float64, requires_grad = True)
y = torch.tensor(y_train, dtype = torch.float64, requires_grad = True)
beta = torch.randn(x.shape[1], 1, dtype = torch.float64, requires_grad = True)

print(beta)

tensor([[-0.4395],
        [ 0.9812]], dtype=torch.float64, requires_grad=True)


In [11]:
# Check if CUDA (GPU) is available
if torch.cuda.is_available():
    dev = "cuda"
else:
    dev = "cpu"

print(f"Running on: {dev}")

# Set up the Adam optimizer
learning_rate = 0.1
optimizer = LBFGS([beta], lr = learning_rate)

# Define a function to calculate the loss and update model parameters
def calculate_loss():
    optimizer.zero_grad()
    value = negative_log_likelihood(beta, x, y)
    value.backward()
    return value

num_iter = 100
for i in range(1, num_iter + 1):
  logl = optimizer.step(calculate_loss)
  logl_value = logl.detach().item()
  estimated_weights = beta.detach().numpy()
  print(f"Iteration {i + 1} || Log-value {logl_value}")
  print(f"Estimated weights: {estimated_weights}")

estimated_weights_LBFGS = beta.detach().numpy()
print("Estimated Parameters (Weights):", estimated_weights_LBFGS)

Running on: cpu
Iteration 2 || Log-value 478.8465576171875
Estimated weights: [[-0.12595029]
 [ 0.9582468 ]]
Iteration 3 || Log-value 478.8465576171875
Estimated weights: [[-0.12595384]
 [ 0.9582568 ]]
Iteration 4 || Log-value 478.8465881347656
Estimated weights: [[-0.1259552 ]
 [ 0.95826036]]
Iteration 5 || Log-value 478.8465576171875
Estimated weights: [[-0.1259567]
 [ 0.9582646]]
Iteration 6 || Log-value 478.8465576171875
Estimated weights: [[-0.12595715]
 [ 0.9582657 ]]
Iteration 7 || Log-value 478.8465576171875
Estimated weights: [[-0.12595783]
 [ 0.95826757]]
Iteration 8 || Log-value 478.8465881347656
Estimated weights: [[-0.1259584]
 [ 0.9582692]]
Iteration 9 || Log-value 478.8465576171875
Estimated weights: [[-0.12595864]
 [ 0.9582699 ]]
Iteration 10 || Log-value 478.8465576171875
Estimated weights: [[-0.12595885]
 [ 0.9582705 ]]
Iteration 11 || Log-value 478.8465576171875
Estimated weights: [[-0.12595904]
 [ 0.958271  ]]
Iteration 12 || Log-value 478.8465576171875
Estimated we

In [12]:
# Convert the variables to torch tensors
x = torch.tensor(x_train, dtype = torch.float16, requires_grad = True)
y = torch.tensor(y_train, dtype = torch.float16, requires_grad = True)
beta = torch.randn(x.shape[1], 1, dtype = torch.float16, requires_grad = True)

In [ ]:
# Check if CUDA (GPU) is available
if torch.cuda.is_available():
    dev = "cuda"
else:
    dev = "cpu"

print(f"Running on: {dev}")

# Set up the Adam optimizer
learning_rate = 0.1
optimizer = LBFGS([beta], lr = learning_rate)

# Define a function to calculate the loss and update model parameters
def calculate_loss():
    optimizer.zero_grad()
    value = negative_log_likelihood(beta, x, y)
    value.backward()
    return value

num_iter = 100
for i in range(1, num_iter + 1):
  logl = optimizer.step(calculate_loss)
  logl_value = logl.detach().item()
  estimated_weights = beta.detach().numpy()
  print(f"Iteration {i + 1} || Log-value {logl_value}")
  print(f"Estimated weights: {estimated_weights}")

estimated_weights_LBFGS = beta.detach().numpy()
print("Estimated Parameters (Weights):", estimated_weights_LBFGS)

In [14]:
import torch

# Function definition
def f(x):
    return x**3 - 3 * x**2 + x

# Values
x_float32 = torch.tensor(1.0, dtype=torch.float32)
x_float64 = torch.tensor(1.0, dtype=torch.float64)

# Derivatives using centered finite difference
delta = 1e-5
f_prime_float32 = (f(x_float32 + delta) - f(x_float32 - delta)) / (2 * delta)
f_prime_float64 = (f(x_float64 + delta) - f(x_float64 - delta)) / (2 * delta)

print("Derivative at x=1 using float32 precision:", f_prime_float32)
print("Derivative at x=1 using float64 precision:", f_prime_float64)

Derivative at x=1 using float32 precision: tensor(-2.0027)
Derivative at x=1 using float64 precision: tensor(-2.0000, dtype=torch.float64)


In [7]:
import torch

# Simulated data
mu = 10  # known mean
sigma_true = 2  # true standard deviation
data_start = torch.normal(mu, sigma_true, size=(10000,))

data32 = data_start
data64 = data_start.to(torch.float64)

In [9]:
# Negative Log Likelihood function
def neg_log_likelihood(sigma, data, mu):
    return -torch.sum(torch.log(1 / (torch.sqrt(2 * torch.tensor(3.1415)) * sigma)) - ((data - mu) ** 2) / (2 * sigma ** 2))

# Parameter to optimize
sigma = torch.tensor(1.0, requires_grad=True, dtype=torch.float64)

# Optimizer
optimizer = torch.optim.SGD([sigma], lr=1)


# Maximum likelihood estimation
num_iterations = 1000
for i in range(num_iterations):
    optimizer.zero_grad()
    loss = neg_log_likelihood(sigma, data32, mu)
    loss.backward()
    optimizer.step()

print("Maximum Likelihood Estimator (sigma):", sigma.item())

Maximum Likelihood Estimator (sigma): 29444.701222334916


In [40]:
print("Maximum Likelihood Estimator (sigma):", sigma.item())

Maximum Likelihood Estimator (sigma): 96554.55653534055


In [3]:
sigma

tensor(26383.0632, dtype=torch.float64, requires_grad=True)

In [16]:
import torch

# Simulated data
mu = 10  # known mean
sigma_true = 3.5  # true standard deviation
data = torch.normal(mu, sigma_true, size=(1000,))

# Negative Log Likelihood function
def neg_log_likelihood(sigma, data, mu):
    return -torch.sum(torch.log(1 / (torch.sqrt(2 * torch.tensor(3.1415)) * sigma)) + ((data - mu) ** 2) / (2 * sigma ** 2))

# Parameter to optimize
sigma = torch.tensor(1.0, requires_grad=True, dtype=torch.float64)

# Optimizer
optimizer = torch.optim.SGD([sigma], lr=0.0000001)

# Maximum likelihood estimation
num_iterations = 1000
for i in range(num_iterations):
    optimizer.zero_grad()
    loss = neg_log_likelihood(sigma, data, mu)
    loss.backward()
    optimizer.step()

print("Maximum Likelihood Estimator (sigma):", sigma.item())

Maximum Likelihood Estimator (sigma): -2.3334010925459867
